In [37]:
#Imports
import pandas as pd
import numpy as np 

In [38]:
# Load the csv files
people_df = pd.read_csv('LDS24 - Data/People.csv')
crashes_df = pd.read_csv('LDS24 - Data/Crashes.csv')
vehicles_df = pd.read_csv('LDS24 - Data/Vehicles.csv')


In [39]:
# Exploration of "People" dataframe

# Print information
print('Info')
people_df.info()


#Print columns
print('Columns')
print(people_df.columns)

#Print shape
print('Shape')
print(people_df.shape)

#Check missing values
print('Missing values')
print(people_df.isna().sum())

columns_with_missing_values = people_df.columns[people_df.isnull().any()].tolist()
print('Columns with missing values')
print(columns_with_missing_values)



Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564565 entries, 0 to 564564
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   PERSON_ID              564565 non-null  object 
 1   PERSON_TYPE            564565 non-null  object 
 2   RD_NO                  564565 non-null  object 
 3   VEHICLE_ID             553869 non-null  float64
 4   CRASH_DATE             564565 non-null  object 
 5   CITY                   420645 non-null  object 
 6   STATE                  422589 non-null  object 
 7   SEX                    557411 non-null  object 
 8   AGE                    403584 non-null  float64
 9   SAFETY_EQUIPMENT       563155 non-null  object 
 10  AIRBAG_DEPLOYED        554113 non-null  object 
 11  EJECTION               557860 non-null  object 
 12  INJURY_CLASSIFICATION  564269 non-null  object 
 13  DRIVER_ACTION          454799 non-null  object 
 14  DRIVER_VISION          454681 n

In [42]:
# Exploration of "Vehicles" dataframe

# Print information
print('Info')
vehicles_df.info()


#Print columns
print('Columns')
print(vehicles_df.columns)

#Print shape
print('Shape')
print(vehicles_df.shape)

#Check missing values
print('Missing values')
print(vehicles_df.isna().sum())

vehicles_columns_with_missing_values = vehicles_df.columns[vehicles_df.isnull().any()].tolist()
print('Columns with missing values')
print(vehicles_columns_with_missing_values)

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460437 entries, 0 to 460436
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CRASH_UNIT_ID        460437 non-null  int64  
 1   RD_NO                460437 non-null  object 
 2   CRASH_DATE           460437 non-null  object 
 3   UNIT_NO              460437 non-null  int64  
 4   UNIT_TYPE            460436 non-null  object 
 5   VEHICLE_ID           450064 non-null  float64
 6   MAKE                 450064 non-null  object 
 7   MODEL                448651 non-null  object 
 8   LIC_PLATE_STATE      410808 non-null  object 
 9   VEHICLE_YEAR         371782 non-null  float64
 10  VEHICLE_DEFECT       450064 non-null  object 
 11  VEHICLE_TYPE         450064 non-null  object 
 12  VEHICLE_USE          450064 non-null  object 
 13  TRAVEL_DIRECTION     450064 non-null  object 
 14  MANEUVER             450064 non-null  object 
 15  OCCUPANT_CNT

In [43]:
# Exploration of "Crashes" dataframe

# Print information
print('Info')
crashes_df.info()


#Print columns
print('Columns')
print(crashes_df.columns)

#Print shape
print('Shape')
print(crashes_df.shape)

#Check missing values
print('Missing values')
print(crashes_df.isna().sum())

crashes_columns_with_missing_values = crashes_df.columns[crashes_df.isnull().any()].tolist()
print('Columns with missing values')
print(crashes_columns_with_missing_values)

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257925 entries, 0 to 257924
Data columns (total 36 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   RD_NO                          257925 non-null  object 
 1   CRASH_DATE                     257925 non-null  object 
 2   POSTED_SPEED_LIMIT             257925 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         257925 non-null  object 
 4   DEVICE_CONDITION               257925 non-null  object 
 5   WEATHER_CONDITION              257925 non-null  object 
 6   LIGHTING_CONDITION             257925 non-null  object 
 7   FIRST_CRASH_TYPE               257925 non-null  object 
 8   TRAFFICWAY_TYPE                257925 non-null  object 
 9   ALIGNMENT                      257925 non-null  object 
 10  ROADWAY_SURFACE_COND           257925 non-null  object 
 11  ROAD_DEFECT                    257925 non-null  object 
 12  REPORT_TYPE              